# Creating A Collada Object

The goal is to produce a basic Collada dae object for display in Google Earth

In [1]:
from collada import *
import simplekml
import numpy

In this section, we outline how to create a collada document from scratch. First, let’s create an empy collada document:

In [2]:
mesh = Collada()

We could save this out, but it would be completely blank. Let’s first add a Material to the document:

In [3]:
mesh = Collada()
effect = material.Effect("effect0", [], "phong", diffuse=(1,0,0), specular=(0,1,0))
mat = material.Material("material0", "mymaterial", effect)
mesh.effects.append(effect)
mesh.materials.append(mat)

Note that the second argument to Effect is for parameters. These are used for textures. We omit textures for simplicity here.

Next, let’s first create some source arrays. These are going to be used to create a triangle set later:

In [4]:
vert_floats = [-50,50,50,
               50,50,50,
               -50,-50,50,
               50,-50,50,
               -50,50,-50,
               50,50,-50,
               -50,-50,-50,
               50,-50,-50]

normal_floats = [0]*72
#[0,0,1,
#                 0,0,1,
#                 0,0,1,
#                 0,0,1,
#                 0,1,0,
#     0,1,0,0,1,0,0,1,0,0,-1,0,0,-1,0,0,-1,0,0,-1,0,-1,0,0,
#     -1,0,0,-1,0,0,-1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,-1,
#     0,0,-1,0,0,-1,0,0,-1]

vert_src = source.FloatSource("cubeverts-array", numpy.array(vert_floats), ('X', 'Y', 'Z'))
normal_src = source.FloatSource("cubenormals-array", numpy.array(normal_floats), ('X', 'Y', 'Z'))

In [5]:
x = numpy.array([[1,2,3],[4,5,6]])
numpy.vstack([x,[7,8,9]])

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [6]:
def create_cylinder(radius=5,height=150,sides=8,offset_x=0,offset_y=0,offset_z=0):
    
    import math
    import numpy as np
    
    angle = 2*math.pi/sides

    vert_floats = []
    
    # create bottom cross-section
    for side in range(sides):
        #print(sector*angle)

        x = radius*math.cos(side*angle)+offset_x
        y = radius*math.sin(side*angle)+offset_y
        z = 0+offset_z

        vert_floats.append(x)
        vert_floats.append(y)
        vert_floats.append(z)

        
    # create top cross-section
    for side in range(sides):
        #print(sector*angle)

        x = radius*math.cos(side*angle)+offset_x
        y = radius*math.sin(side*angle)+offset_y
        z = height+offset_z

        vert_floats.append(x)
        vert_floats.append(y)
        vert_floats.append(z)

    #print(vert_floats)

    
    # define triangles
    indices = []
    for side in range(sides):
        tri1 = (0+side)%sides
        tri2 = (1+side)%sides
        tri3 = sides+side

        indices.append(tri1)
        indices.append(int(len(indices)/2-0.5))
        indices.append(tri2)
        indices.append(int(len(indices)/2-0.5))
        indices.append(tri3)
        indices.append(int(len(indices)/2-0.5))

        tri4 = sides+(side+1)%sides

        indices.append(tri2)
        indices.append(int(len(indices)/2-0.5))
        indices.append(tri3)
        indices.append(int(len(indices)/2-0.5))
        indices.append(tri4)
        indices.append(int(len(indices)/2-0.5))

    #print(indices)

    return vert_floats,indices


In [7]:
def list_adder(lst,value):
    
    new_list = [item + value for item in lst]
    
    return new_list


In [8]:
# create tower
tower_vert_floats,tower_indices = create_cylinder(radius=5,height=150,sides=8,offset_x=0,offset_y=0,offset_z=0)

# create nacelle
nacelle_vert_floats,nacelle_indices = create_cylinder(radius=10,height=10,sides=4,offset_x=0,offset_y=0,offset_z=150)

nacelle_indices = list_adder(nacelle_indices,int(len(tower_vert_floats)/3))

vert_floats = tower_vert_floats + nacelle_vert_floats
indices = tower_indices + nacelle_indices



normal_floats = [0]*len(vert_floats)*3

vert_src = source.FloatSource("cubeverts-array", numpy.array(vert_floats), ('X', 'Y', 'Z'))
normal_src = source.FloatSource("cubenormals-array", numpy.array(normal_floats), ('X', 'Y', 'Z'))

geom = geometry.Geometry(mesh, "geometry0", "mycube", [vert_src, normal_src])

input_list = source.InputList()
input_list.addInput(0, 'VERTEX', "#cubeverts-array")
input_list.addInput(1, 'NORMAL', "#cubenormals-array")

indices = numpy.array(indices)

triset = geom.createTriangleSet(indices, input_list, "materialref")
geom.primitives.append(triset)
mesh.geometries.append(geom)

matnode = scene.MaterialNode("materialref", mat, inputs=[])
geomnode = scene.GeometryNode(geom, [matnode])
node = scene.Node("node0", children=[geomnode])

myscene = scene.Scene("myscene", [node])
mesh.scenes.append(myscene)
mesh.scene = myscene

mesh.write('tmp/collada02.dae')

In [13]:
a=numpy.array(tower_indices + nacelle_indices)

In [14]:
a

array([ 0,  0,  1,  1,  8,  2,  1,  3,  8,  4,  9,  5,  1,  6,  2,  7,  9,
        8,  2,  9,  9, 10, 10, 11,  2, 12,  3, 13, 10, 14,  3, 15, 10, 16,
       11, 17,  3, 18,  4, 19, 11, 20,  4, 21, 11, 22, 12, 23,  4, 24,  5,
       25, 12, 26,  5, 27, 12, 28, 13, 29,  5, 30,  6, 31, 13, 32,  6, 33,
       13, 34, 14, 35,  6, 36,  7, 37, 14, 38,  7, 39, 14, 40, 15, 41,  7,
       42,  0, 43, 15, 44,  0, 45, 15, 46,  8, 47, 16, 16, 17, 17, 20, 18,
       17, 19, 20, 20, 21, 21, 17, 22, 18, 23, 21, 24, 18, 25, 21, 26, 22,
       27, 18, 28, 19, 29, 22, 30, 19, 31, 22, 32, 23, 33, 19, 34, 16, 35,
       23, 36, 16, 37, 23, 38, 20, 39])